In [83]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline import Pipeline  # Use the imblearn pipeline

from sklearn.utils.validation import column_or_1d
from sklearn.ensemble import RandomForestClassifier

In [84]:
%run ./ml_model_init.ipynb

## Functions


In [85]:
# For the linear model
def Linear_regression(features_train, features_pred, target_train):
    # bool_cols = features_train.drop(columns=['was_home']).columns.tolist()

    bool_cols = features_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = features_train.select_dtypes(include=['object', 'category']).columns.tolist()

    print('================>', bool_cols, categorical_cols)
    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('one_hot_encoder', OneHotEncoder())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, bool_cols),
            ('cat', categorical_transformer, categorical_cols),
        ])

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', LinearRegression())
    ])

    model = TransformedTargetRegressor(regressor=pipeline, transformer=StandardScaler())
    model.fit(features_train, target_train)

    return model.predict(features_pred)

# Decision Tree Model
def DecisionTreeRegression(features_train, features_pred, target_train):
    bool_cols = features_train.drop(columns=['was_home']).columns.tolist()
    categorical_cols = ['was_home']

    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('one_hot_encoder', OneHotEncoder())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, bool_cols),
            ('cat', categorical_transformer, categorical_cols),
        ])

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', DecisionTreeRegressor())
    ])

    model = TransformedTargetRegressor(regressor=pipeline, transformer=StandardScaler())
    model.fit(features_train, target_train)

    return model.predict(features_pred)

# RandomForestRegressor
def RandomForestRegression(features_train, features_pred, target_train, hyperparameters):
    model = TransformedTargetRegressor(RandomForestRegressor(
        n_estimators=hyperparameters['n_estimators'],  max_depth=hyperparameters['max_depth'], criterion=hyperparameters['criterion'], random_state=18), transformer=StandardScaler())
    model.fit(features_train, target_train)

    pred_pred = model.predict(features_pred)

def XGBoostRegression(features_train, features_pred, target_train, hyperparameters):
    regressor = xgb(learning_rate=hyperparameters["learning_rate"],
                    n_estimators=hyperparameters["n_estimators"],
                    max_depth=hyperparameters["max_depth"],
                    eval_metric='rmsle')

    model = TransformedTargetRegressor(regressor, transformer=StandardScaler())


    model.fit(features_train, target_train)

    return model.predict(features_pred)


def Random_Forest_Classifier(features_train, features_pred, target_train, hy_params):
    encoder = LabelEncoder()
    cs_train_ = encoder.fit_transform(target_train)

    # bool_cols = features_train.drop(columns=['was_home']).columns.tolist()
    # categorical_cols = ['was_home']
    bool_cols = features_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = features_train.select_dtypes(include=['object', 'category']).columns.tolist()

    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('one_hot_encoder', OneHotEncoder(sparse_output=False)),
        # ('to_dense', ToDense())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, bool_cols),
            ('cat', categorical_transformer, categorical_cols),
        ])

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote', BorderlineSMOTE(sampling_strategy='auto', random_state=42)),  # Apply SMOTE to the data
        ('classifier', RandomForestClassifier(bootstrap = hy_params['bootstrap'], min_samples_split = hy_params['min_samples_split'], n_estimators =hy_params['n_estimators']  , class_weight='balanced', random_state=42))  # Random Forest Classifier
    ])

    pipeline.fit(features_train, cs_train_)

    # Make predictions
    return pipeline.predict(features_pred)

In [86]:
data_tar_preds = data_tar_.copy()

## Goals Predictor


In [87]:

# (data['minutes_5'] >= 300) &
xg_data= data[(data['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'ict_index_3',  'ict_index_5',
                                                                     'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5', 'was_home', 'xG', 'xG_3',
                                                                      'xG_5', 'expected_goals_3', 'expected_goals_5', 'goals_scored_3', 'whh', 'whd', 'wha', 'value']].copy()
xg_data = xg_data.dropna()
# (data_tar['minutes_5'] >= 300) &
xg_data_tar = data_tar_[ (data_tar_['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'ict_index_3',
                                                                                         'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3',
                                                                                        'threat_5', 'was_home', 'xG', 'xG_3', 'xG_5', 'expected_goals_3', 'expected_goals_5',
                                                                                        'goals_scored_3','whh', 'whd', 'wha', 'value']].copy()

In [88]:

xg = xg_data[['xG']].copy()
feats = xg_data.drop('xG', axis=1)

xg_tar = xg_data_tar[['xG']].copy()
feats_tar = xg_data_tar.drop('xG', axis=1)


In [89]:
feats_tar.isnull().sum()

position             0
minutes_3            0
minutes_5            0
team_h_difficulty    0
team_a_difficulty    0
ict_index_3          0
ict_index_5          0
influence_3          0
influence_5          0
creativity_3         0
creativity_5         0
threat_3             0
threat_5             0
was_home             0
xG_3                 0
xG_5                 0
expected_goals_3     0
expected_goals_5     0
goals_scored_3       0
whh                  0
whd                  0
wha                  0
value                0
dtype: int64

In [90]:
xg_pred = Linear_regression(feats, feats_tar, xg)
xg_tar['xg_pred'] = xg_pred
# xg_tar = xg_tar.drop('xG', axis=1)
# comb_data = xg_data_tar.join(xg_tar)
# comb_data

def add_preds(row):
    if row.name in list(xg_tar.index):
        # print(row.name, xg_tar.loc[row.name, 'xg_pred'])
        row['xg_pred'] = xg_tar.loc[row.name, 'xg_pred']
    else:
        row['xg_pred'] = 0
    return row
data_tar_preds = data_tar_preds.apply(add_preds, axis=1)
data_tar_preds['xg_pred']

================> ['minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'ict_index_3', 'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5', 'xG_3', 'xG_5', 'expected_goals_3', 'expected_goals_5', 'goals_scored_3', 'whh', 'whd', 'wha', 'value'] ['position']


12     -0.139593
19     -0.136064
43     -0.087085
52     -0.122851
71     -0.132256
          ...   
8119   -0.123291
8122   -0.136502
8128   -0.125056
8155   -0.098733
8182    0.123843
Name: xg_pred, Length: 517, dtype: float64

## Assists Predictor


In [91]:

xa_data= data[ (data['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'ict_index_3',  'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5',
                                                                         'threat_3', 'threat_5', 'was_home', 'expected_assists', 'xA_3', 'xA_5',  'expected_assists_3', 'expected_assists_5', 'whh', 'whd', 'wha']].copy()

xa_data_tar = data_tar_[(data_tar_['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'ict_index_3',  'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5',
                                                                         'threat_3', 'threat_5','was_home', 'expected_assists', 'xA_3', 'xA_5',  'expected_assists_3',
                    'expected_assists_5', 'whh', 'whd', 'wha']].copy()

xa_data = xa_data.dropna()

In [92]:

xa = xa_data[['expected_assists']].copy()
feats = xa_data.drop('expected_assists', axis=1)

xa_tar = xa_data_tar[['expected_assists']].copy()
feats_tar = xa_data_tar.drop('expected_assists', axis=1)

In [93]:
xa_pred = Linear_regression(feats, feats_tar, xa)
xa_tar['xa_pred'] = xa_pred
# xa_tar = xa_tar.drop('expected_assists', axis=1)
# comb_data = comb_data.join(xa_tar, rsuffix='x')
# comb_data

def add_preds(row):
    if row.name in list(xa_tar.index):
        # print(row.name, xa_tar.loc[row.name, 'xa_pred'])
        row['xa_pred'] = xa_tar.loc[row.name, 'xa_pred']
    else:
        row['xa_pred'] = 0
    return row
data_tar_preds = data_tar_preds.apply(add_preds, axis=1)
data_tar_preds['xa_pred']

================> ['minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'ict_index_3', 'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5', 'xA_3', 'xA_5', 'expected_assists_3', 'expected_assists_5', 'whh', 'whd', 'wha'] ['position']


12      0.051849
19      0.037616
43      0.070582
52      0.049594
71      0.037000
          ...   
8119    0.088287
8122    0.037635
8128    0.049594
8155    0.109230
8182    0.189518
Name: xa_pred, Length: 517, dtype: float64

## Clean Sheets


In [94]:
cs_data= data[(data['position'] != 'FWD')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'clean_sheets', 'clean_sheets_3', 'clean_sheets_5',
                                                                         'expected_goals_conceded_3', 'expected_goals_conceded_5', 'ict_index_3',  'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5',
                                                                         'threat_3', 'threat_5', 'whh', 'whd', 'wha']]

cs_data_tar = data_tar_[ (data_tar_['position'] != 'FWD')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'clean_sheets', 'clean_sheets_3', 'clean_sheets_5',
                                                                         'expected_goals_conceded_3', 'expected_goals_conceded_5', 'ict_index_3',  'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5',
                                                                         'threat_3', 'threat_5', 'whh', 'whd', 'wha']]

cs_data = cs_data.dropna()

In [95]:
cs = cs_data[['clean_sheets']]
feats = cs_data.drop('clean_sheets', axis=1)

cs_tar = cs_data_tar[['clean_sheets']].copy()
feats_tar = cs_data_tar.drop('clean_sheets', axis=1)

In [96]:
feats.isnull().sum()

position                     0
minutes_3                    0
minutes_5                    0
team_h_difficulty            0
team_a_difficulty            0
was_home                     0
clean_sheets_3               0
clean_sheets_5               0
expected_goals_conceded_3    0
expected_goals_conceded_5    0
ict_index_3                  0
ict_index_5                  0
influence_3                  0
influence_5                  0
creativity_3                 0
creativity_5                 0
threat_3                     0
threat_5                     0
whh                          0
whd                          0
wha                          0
dtype: int64

In [97]:
cs_hy_params = {'bootstrap': False, 'min_samples_split': 2, 'n_estimators': 100}
cs_pred = Random_Forest_Classifier(feats, feats_tar, column_or_1d(cs), cs_hy_params)
cs_tar['cs_pred'] = cs_pred
# cs_tar = cs_tar.drop('clean_sheets', axis=1)
# comb_data = comb_data.join(cs_tar, rsuffix='x')
# comb_data

def add_preds(row):
    if row.name in list(cs_tar.index):
        # print(row.name, cs_tar.loc[row.name, 'cs_pred'])
        row['cs_pred'] = cs_tar.loc[row.name, 'cs_pred']
    else:
        row['cs_pred'] = 0
    return row
data_tar_preds = data_tar_preds.apply(add_preds, axis=1)
data_tar_preds['cs_pred']

12      0
19      0
43      0
52      0
71      0
       ..
8119    0
8122    1
8128    0
8155    0
8182    0
Name: cs_pred, Length: 517, dtype: int64

## Yellow Cards


In [98]:
#
yc_data= data[ (data['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'ict_index_3',
                                                                         'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5',
                                                                         'yellow_cards', 'yellow_cards_3', 'yellow_cards_5', 'whh', 'whd', 'wha']]

yc_data_tar = data_tar_[ (data_tar_['position'] != 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'ict_index_3',
                                                                         'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5',
                                                                         'yellow_cards', 'yellow_cards_3', 'yellow_cards_5', 'whh', 'whd', 'wha']]


yc_data = yc_data.dropna()

In [99]:
yc = yc_data[['yellow_cards']]
feats = yc_data.drop('yellow_cards', axis=1)

yc_tar = yc_data_tar[['yellow_cards']].copy()
feats_tar = yc_data_tar.drop('yellow_cards', axis=1)

In [100]:
params =  {'bootstrap': False, 'min_samples_split': 10, 'n_estimators': 300}
yc_pred = Random_Forest_Classifier(feats, feats_tar, column_or_1d(yc), params)
yc_tar['yc_pred'] = yc_pred
# yc_tar = yc_tar.drop('yellow_cards', axis=1)
# comb_data = comb_data.merge(yc_tar, left_index=True, right_index=True, how='inner')
# comb_data

def add_preds(row):
    if row.name in list(yc_tar.index):
        # print(row.name, yc_tar.loc[row.name, 'yc_pred'])
        row['yc_pred'] = yc_tar.loc[row.name, 'yc_pred']
    else:
        row['yc_pred'] = 0
    return row
data_tar_preds = data_tar_preds.apply(add_preds, axis=1)
data_tar_preds['yc_pred']

12      0
19      0
43      0
52      0
71      0
       ..
8119    0
8122    0
8128    0
8155    0
8182    0
Name: yc_pred, Length: 517, dtype: int64

## Saves


In [101]:
sv_data= data[ (data['position'] == 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'ict_index_3',
                                                                         'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5',
                                                                         'clean_sheets_3', 'clean_sheets_5', 'expected_goals_conceded_3', 'expected_goals_conceded_5','saves',
                                                                          'saves_3', 'saves_5', 'whh', 'whd', 'wha']]

sv_data_tar = data_tar_[(data_tar_['position'] == 'GK')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'ict_index_3',
                                                                         'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5',
                                                                         'clean_sheets_3', 'clean_sheets_5', 'expected_goals_conceded_3', 'expected_goals_conceded_5','saves',
                                                                          'saves_3', 'saves_5', 'whh', 'whd', 'wha']]

sv_data = sv_data.dropna()

In [102]:
sv = sv_data[['saves']]
feats = sv_data.drop(['saves', 'position'], axis=1)

sv_tar = sv_data_tar[['saves']].copy()
feats_tar = sv_data_tar.drop(['saves', 'position'], axis=1)

In [103]:
sv_pred = Linear_regression(feats, feats_tar, sv)
sv_tar['sv_pred'] = sv_pred
# sv_tar = sv_tar.drop('saves', axis=1)
# comb_data = comb_data.join(sv_tar, rsuffix='x')
# comb_data

def add_preds(row):
    if row.name in list(sv_tar.index):
        # print(row.name, sv_tar.loc[row.name, 'sv_pred'])
        row['sv_pred'] = sv_tar.loc[row.name, 'sv_pred']
    else:
        row['sv_pred'] = 0
    return row
data_tar_preds = data_tar_preds.apply(add_preds, axis=1)
data_tar_preds['sv_pred']

================> ['minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'ict_index_3', 'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5', 'threat_3', 'threat_5', 'clean_sheets_3', 'clean_sheets_5', 'expected_goals_conceded_3', 'expected_goals_conceded_5', 'saves_3', 'saves_5', 'whh', 'whd', 'wha'] []


12      0.0
19      0.0
43      0.0
52      0.0
71      0.0
       ... 
8119    0.0
8122    0.0
8128    0.0
8155    0.0
8182    0.0
Name: sv_pred, Length: 517, dtype: float64

## Total Points


In [104]:
# For playing up to 60 minutes	1
# For playing 60 minutes or more (excluding stoppage time)	2
# For each goal scored by a goalkeeper	10
# For each goal scored by a defender	6
# For each goal scored by a midfielder	5
# For each goal scored by a forward	4
# For each goal assist	3
# For a clean sheet by a goalkeeper or defender	4
# For a clean sheet by a midfielder	1
# For every 3 shot saves by a goalkeeper	1
# For each penalty save	5
# For each penalty miss	-2
# Bonus points for the best players in a match	1-3
# For every 2 goals conceded by a goalkeeper or defender	-1
# For each yellow card	-1
# For each red card	-3
# For each own goal	-2

## GK
## ---> >60  = 2
## ---> cs*4
## ---> ((sv)/3)*1
## ---> gc
## ---> yc*-1


## DEF
## ---> >60  = 2
## ---> xg*6
## ---> xa*3
## ---> cs*4
## ---> gc
## ---> yc*-1


## MID
## ---> >60  = +2
## ---> xg*5
## ---> xa*3
## ---> cs*1
## ---> yc*-1


## FWD
## ---> >60  = 2
## ---> xg*4
## ---> xa*3
## ---> yc*-1

In [105]:
def calc_total_pts(row):
    if(row.position == 'GK'):
                ## ---> >60  = 2
        ## ---> cs*4
        ## ---> ((sv)/3)*1
        ## ---> gc
        ## ---> yc*-1
        row['xPts'] = 2 + row['cs_pred']*4 + ((row['sv_pred'])/3)*1 + row['yc_pred']*(-1)

    elif(row.position == 'DEF'):
        ## ---> >60  = 2
        ## ---> xg*6
        ## ---> xa*3
        ## ---> cs*4
        ## ---> gc
        ## ---> yc*-1
        row['xPts'] = 2 + row['xg_pred']*6 + row['xa_pred']*3 + row['cs_pred']*4 + row['yc_pred']*(-1)
    elif(row.position == 'MID'):
        ## MID
        ## ---> >60  = +2
        ## ---> xg*5
        ## ---> xa*3
        ## ---> cs*1
        ## ---> yc*-1

        row['xPts'] = 2 + row['xg_pred']*5 + row['xa_pred']*3 + row['cs_pred']*1 + row['yc_pred']*(-1)

    elif(row.position == 'FWD'):
        ## ---> >60  = 2
        ## ---> xg*4
        ## ---> xa*3
        ## ---> yc*-1

        row['xPts'] = 2 + row['xg_pred']*4 + row['xa_pred']*3 + row['yc_pred']*(-1)

    return row

data_tar_preds = data_tar_preds.apply(calc_total_pts, axis=1)
data_tar_preds

,assists_x,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,...,pts_bps,whh,whd,wha,xg_pred,xa_pred,cs_pred,yc_pred,sv_pred,xPts
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.740,0.174,0.087,-0.139593,0.051849,0,0,0.0,1.457581
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.278,0.249,0.473,-0.136064,0.037616,0,0,0.0,1.296463
43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.278,0.249,0.473,-0.087085,0.070582,0,0,0.0,1.776318
52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.278,0.249,0.473,-0.122851,0.049594,0,0,0.0,1.534527
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.278,0.249,0.473,-0.132256,0.037000,0,0,0.0,1.317467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.278,0.249,0.473,-0.123291,0.088287,0,0,0.0,1.648407
8122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.278,0.249,0.473,-0.136502,0.037635,1,0,0.0,5.293893
8128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.278,0.249,0.473,-0.125056,0.049594,0,0,0.0,1.523503
8155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.278,0.249,0.473,-0.098733,0.109230,0,0,0.0,1.834026


## Player Points


In [106]:
data_tar_preds[['element', 'position', 'xPts']]

,element,position,xPts
12,NaN,MID,1.457581
19,NaN,DEF,1.296463
43,NaN,MID,1.776318
52,NaN,MID,1.534527
71,NaN,DEF,1.317467
...,...,...,...
8119,NaN,MID,1.648407
8122,NaN,DEF,5.293893
8128,NaN,MID,1.523503
8155,NaN,MID,1.834026


In [108]:
data_tar_preds.to_csv('./predicted.csv', index=False)